In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [28]:
import numpy as np
import pandas as pd

final_df = pd.read_excel('final_df.xlsx')

final_df = final_df[['Title', 'Author', 'Edition', 'Reviews','Ratings','Synopsis','Genre','BookCategory','Price']]
final_df.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62


In [29]:
final_df = final_df.drop(['Edition'], axis = 1) 
final_df = final_df.drop(['Synopsis'], axis = 1) 

final_df.head()

,Title,Author,Reviews,Ratings,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,4.8 out of 5 stars,6 customer reviews,International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography",965.62


In [30]:
X = final_df.iloc[:,:].values
type(X[:,2])

numpy.ndarray

In [0]:
import re
for i in range (0,7797):
  review_col = re.findall(r'\d\W\d', X[:,2][i])
  review_col = float(review_col[0])
  X[:,2][i] = review_col
  
for i in range (0,7797):
  ratings_col = re.findall(r'\d+', X[:,3][i])
  ratings_col = int(ratings_col[0])
  X[:,3][i] = ratings_col
  

In [32]:
X

array([["The Prisoner's Gold (The Hunters 3)", 'Chris Kuzneski', 4.0,
        ..., 'Action & Adventure (Books)', 'Action & Adventure', 220.0],
       ['Guru Dutt: A Tragedy in Three Acts', 'Arun Khopkar', 3.9, ...,
        'Cinema & Broadcast (Books)',
        'Biographies, Diaries & True Accounts', 202.93],
       ['Leviathan (Penguin Classics)', 'Thomas Hobbes', 4.8, ...,
        'International Relations', 'Humour', 299.0],
       ...,
       ['The Kite Runner Graphic Novel', 'Khaled Hosseini', 4.0, ...,
        'Humour (Books)', 'Humour', nan],
       ['Panzer Leader (Penguin World War II Collection)',
        'Heinz Guderian', 3.5, ..., 'United States History',
        'Biographies, Diaries & True Accounts', nan],
       ['Complete Spanish Step-by-Step', 'Barbara Bregstein', 4.5, ...,
        'Dictionaries', 'Language, Linguistics & Writing', nan]],
      dtype=object)

In [0]:
import pickle
pickle_out = open("Done_with_executing_re_for_columns_with+genre.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

In [0]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder()
X[:,0]=le.fit_transform(X[:,0])
X[:,1]=le.fit_transform(X[:,1])
X[:,4]=le.fit_transform(X[:,4])
X[:,5]=le.fit_transform(X[:,5])

X = X[:,:-1]

In [35]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(X).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [36]:
train = X[:6237,:]
print(train.shape)

test = X[6237:,:]
print(test.shape)
type(train)

(6237, 11883)
(1560, 11883)


numpy.ndarray

In [39]:
y = final_df.iloc[:,6].values
y = y[:6237]
y.shape
y

array([220.  , 202.93, 299.  , ..., 399.  , 319.  , 452.  ])

In [40]:
# Fitting Random Forest Regression to the dataset 
# import the regressor 
from sklearn.ensemble import RandomForestRegressor 

# create regressor object 
regressor = RandomForestRegressor(n_estimators = 400, random_state = 0, n_jobs = -1, max_features = "auto") 

# fit the regressor with x and y data 
regressor.fit(train, y) 


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [41]:
prediction = regressor.predict(test)
submission = pd.DataFrame({'Price':prediction})
filename = 'prediction_books.xlsx'
submission.to_excel(filename,index=False)
print('Saved file: ' + filename)

Saved file: prediction_books.xlsx


In [26]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor , X = train , y = y , cv = 10 , n_jobs=-1)
accuracies.mean()
accuracies.std()
accuracies

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


array([0.08337359, 0.04705645, 0.08591099, 0.02234639, 0.05574818,
       0.0921135 , 0.09109865, 0.07756046, 0.0303429 , 0.06564548])

In [27]:
print(accuracies.mean())

0.0651196592810021
